# Introduction

Here I Will use all the function that I create in the previous notebook but the hibrid model would have now a LSTM! 

I have taking lot of ideas from this kaggle: https://www.kaggle.com/code/dimitreoliveira/time-series-forecasting-with-lstm-autoencoders/notebook

As always I change a little the original code to make it more efficient. Namely, in the code above the input it is created using a loop that is very innefficient. 

In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression 
import plotly.express as px
import seaborn as sns
from datetime import datetime


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from tensorflow.keras import optimizers, Sequential, Model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

import os

In [2]:
import plotly.io as pio


pio.renderers.default = "jpg"

import os

In [3]:
os.environ["KERAS_BACKEND"] = "tensorflow"

In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
# Read the data
# This data you can find here: https://www.kaggle.com/c/home-data-for-ml-course/data
data_dir ="sales_train"

train = pd.read_csv('sales_train.csv',
        parse_dates=['date'],
        dtype={'date': 'str', 'date_block_num': 'int32', 'shop_id': 'int32', 
        'item_id': 'int32', 'item_price': 'float32', 'item_cnt_day': 'int32'}
)
train["date"] = pd.to_datetime(train["date"], format='%d.%m.%Y')

In [6]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.000000,1
1,2013-01-03,0,25,2552,899.000000,1
2,2013-01-05,0,25,2552,899.000000,-1
3,2013-01-06,0,25,2554,1709.050049,1
4,2013-01-15,0,25,2555,1099.000000,1


In [7]:
train.item_cnt_day.describe()

count    2.935849e+06
mean     1.242641e+00
std      2.618834e+00
min     -2.200000e+01
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      2.169000e+03
Name: item_cnt_day, dtype: float64

In [8]:
train.item_cnt_day.quantile(0.999)

22.0

In [9]:
# it seems there is an outlier let us see...

In [10]:
max_value = max(train.item_cnt_day)
max_value

2169

In [11]:
train[train['item_cnt_day']<max_value].item_cnt_day.describe()

count    2.935848e+06
mean     1.241903e+00
std      2.292962e+00
min     -2.200000e+01
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+03
Name: item_cnt_day, dtype: float64

It is evident there are some outliers because 99,9% of values are less than 22. Let us keep these!

In [12]:
train_monthly = train[['date', 'date_block_num', 'shop_id', 'item_id', 'item_cnt_day']]
train_monthly = train_monthly.sort_values('date').groupby(['date_block_num', 'shop_id', 'item_id'], as_index=False)
train_monthly = train_monthly.agg({'item_cnt_day':['sum']})
train_monthly.columns = ['date_block_num', 'shop_id', 'item_id', 'item_cnt']
# IN the kaggle there is this query, but the explanation is not good enough to keep it
train_monthly = train_monthly.query('item_cnt >= 0 and item_cnt <= 22')
# Label
train_monthly['item_cnt_month'] = train_monthly.sort_values('date_block_num').groupby(['shop_id', 'item_id'])['item_cnt'].shift(-1)

display(train_monthly.head(4))
#display(train_monthly.describe())

,date_block_num,shop_id,item_id,item_cnt,item_cnt_month
0,0,0,32,6,10.0
1,0,0,33,3,3.0
2,0,0,35,1,14.0
3,0,0,43,1,NaN


In [13]:
monthly_series = train_monthly.pivot_table(index=['shop_id', 'item_id'], columns='date_block_num',values='item_cnt', fill_value=int(0)).reset_index()
monthly_series.head()

date_block_num,shop_id,item_id,0,1,2,3,4,5,6,7,...,24,25,26,27,28,29,30,31,32,33
0,0,31,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,32,6.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,33,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,35,1.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,36,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Lazy trick to change all columns to integer
for col in monthly_series.columns:
    try:
        monthly_series[col] = monthly_series[col].apply(lambda x:int(x))
    except:
        pass

In [15]:
temp_columns1 = [a for a in range(21,34)]
column1 = ["shop_id","item_id"] + temp_columns1 
temp_columns2 = [a for a in range(20,33)]
column2 = ["shop_id","item_id"] + temp_columns2 

In [16]:
monthly_series_new1 = monthly_series[column1]
monthly_series_new2 = monthly_series[column2]
column_names = [a for a in range(0,13)]
monthly_series_new1.columns = ["shop_id","item_id"] + column_names
monthly_series_new2.columns = ["shop_id","item_id"] + column_names
#monthly_series_new2[monthly_series_new2["shop_id"]==2].head()


In [17]:
#monthly_series_new1[monthly_series_new1["shop_id"]==2]

Something good to mention, notice that we have 13 months so the "0" and the "12" column are indeed the same month! 

In [18]:
data_series = pd.concat([monthly_series_new1,monthly_series_new2], axis=0)
data_series.head()

,shop_id,item_id,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,31,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,32,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,33,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,35,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,36,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
print("Train set", data_series.shape)

Train set (847560, 15)


In [20]:
# to speed things up I am going to bluntly take out all rows that just have zero.
# the justification for that is probably those item in that shop are discontinue
data_series = data_series.loc[~(data_series[column_names]==0).all(axis=1)]

In [21]:
data_series.rename(columns={12: "label"},inplace=True)

In [22]:
data_series = data_series.drop(['item_id', 'shop_id'], axis=1)

In [23]:
labels = data_series['label']
data_series.drop('label', axis=1, inplace=True)
train, valid, Y_train, Y_valid = train_test_split(data_series, labels.values, test_size=0.10, random_state=0)

In [24]:
X_train = train.values.reshape((train.shape[0], train.shape[1], 1))
X_valid = valid.values.reshape((valid.shape[0], valid.shape[1], 1))

print("Train set reshaped", X_train.shape)
print("Validation set reshaped", X_valid.shape)

Train set reshaped (321368, 12, 1)
Validation set reshaped (35708, 12, 1)


In [25]:
serie_size =  X_train.shape[1] # 12
n_features =  X_train.shape[2] # 1

epochs = 15
batch = 128
lr = 0.0005

lstm_model = Sequential()
lstm_model.add(LSTM(10, input_shape=(serie_size, n_features), return_sequences=True))
lstm_model.add(LSTM(6, activation='relu', return_sequences=True))
lstm_model.add(LSTM(1, activation='relu'))
lstm_model.add(Dense(15, kernel_initializer='glorot_normal', activation='relu'))
lstm_model.add(Dense(15, kernel_initializer='glorot_normal', activation='relu'))
lstm_model.add(Dense(1))
lstm_model.summary()

adam = optimizers.Adam(lr)
lstm_model.compile(loss='mse', optimizer=adam)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 12, 10)            480       
                                                                 
 lstm_1 (LSTM)               (None, 12, 6)             408       
                                                                 
 lstm_2 (LSTM)               (None, 1)                 32        
                                                                 
 dense (Dense)               (None, 15)                30        
                                                                 
 dense_1 (Dense)             (None, 15)                240       
                                                                 
 dense_2 (Dense)             (None, 1)                 16        
                                                                 
Total params: 1,206
Trainable params: 1,206
Non-trainabl

In [26]:
lstm_history = lstm_model.fit(X_train, Y_train, 
                              validation_data=(X_valid, Y_valid), 
                              batch_size=batch, 
                              epochs=epochs, 
                              verbose=2)

Epoch 1/15
2511/2511 - 292s - loss: 0.9358 - val_loss: 0.7577 - 292s/epoch - 116ms/step
Epoch 2/15
2511/2511 - 290s - loss: 0.7964 - val_loss: 0.7147 - 290s/epoch - 115ms/step
Epoch 3/15
2511/2511 - 287s - loss: 0.7793 - val_loss: 0.7134 - 287s/epoch - 114ms/step
Epoch 4/15
2511/2511 - 287s - loss: 0.7771 - val_loss: 0.7180 - 287s/epoch - 114ms/step
Epoch 5/15
2511/2511 - 293s - loss: 0.7761 - val_loss: 0.7116 - 293s/epoch - 117ms/step
Epoch 6/15
2511/2511 - 292s - loss: 0.7740 - val_loss: 0.7105 - 292s/epoch - 116ms/step
Epoch 7/15
2511/2511 - 292s - loss: 0.7750 - val_loss: 0.7142 - 292s/epoch - 116ms/step
Epoch 8/15
2511/2511 - 294s - loss: 0.7738 - val_loss: 0.7098 - 294s/epoch - 117ms/step
Epoch 9/15
2511/2511 - 291s - loss: 0.7738 - val_loss: 0.7131 - 291s/epoch - 116ms/step
Epoch 10/15
2511/2511 - 247s - loss: 0.7738 - val_loss: 0.7109 - 247s/epoch - 98ms/step
Epoch 11/15
2511/2511 - 202s - loss: 0.7735 - val_loss: 0.7094 - 202s/epoch - 80ms/step
Epoch 12/15
2511/2511 - 200s - l

In [27]:
import pickle
lstm_model.save('keras/lstm2.keras')
with open('lstm_train_history_dict2', 'wb') as file_pi:
    pickle.dump(lstm_history.history, file_pi)

In [28]:
# loading
with open('lstm_train_history_dict2', "rb") as file_pi:
    history = pickle.load(file_pi)
#model = keras.models.load_model('keras/lstm2.keras')

In [29]:
#history

In [30]:
valid[10:25]

,0,1,2,3,4,5,6,7,8,9,10,11
166027,0,0,3,0,0,1,0,0,0,0,0,0
253702,1,0,1,1,0,0,0,0,0,0,0,0
132797,0,0,0,0,0,0,1,0,0,1,0,0
361975,1,0,2,0,1,0,0,0,0,0,0,0
234582,0,0,0,1,1,0,0,0,0,0,0,0
14676,1,1,0,0,0,0,0,0,0,0,0,0
307620,0,0,0,0,0,0,0,0,0,0,0,1
248779,5,7,9,12,3,6,1,3,3,2,1,2
131071,0,0,1,0,0,0,0,0,0,0,0,0
236594,0,0,0,0,1,0,0,0,0,0,0,0


In [36]:
pred = lstm_model.predict(valid[:50])
pred

2/2 [==============================] - 0s 27ms/step


array([[0.05092265],
       [0.03874768],
       [0.13935888],
       [0.18826103],
       [1.814379  ],
       [0.03614251],
       [0.03844588],
       [0.03433517],
       [0.13267794],
       [0.3063698 ],
       [0.05142922],
       [0.04405057],
       [0.12351363],
       [0.05120927],
       [0.03538565],
       [0.03963808],
       [0.3044494 ],
       [1.8568528 ],
       [0.0322932 ],
       [0.03433517],
       [1.0374655 ],
       [0.04446816],
       [0.3044494 ],
       [0.06772707],
       [0.31055993],
       [0.03215153],
       [0.29152378],
       [0.16666147],
       [0.03419349],
       [0.0322932 ],
       [0.51841253],
       [0.19386658],
       [0.48419002],
       [0.04550255],
       [0.04735994],
       [0.0322932 ],
       [0.03419349],
       [0.06452044],
       [0.18095762],
       [0.09562363],
       [0.05135094],
       [0.03419349],
       [0.32851875],
       [0.03963808],
       [0.05198386],
       [0.03433517],
       [0.07830214],
       [0.087

In [37]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [41]:
Y_valid[:50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int64)

In [42]:
pred.flatten()

array([0.05092265, 0.03874768, 0.13935888, 0.18826103, 1.814379  ,
       0.03614251, 0.03844588, 0.03433517, 0.13267794, 0.3063698 ,
       0.05142922, 0.04405057, 0.12351363, 0.05120927, 0.03538565,
       0.03963808, 0.3044494 , 1.8568528 , 0.0322932 , 0.03433517,
       1.0374655 , 0.04446816, 0.3044494 , 0.06772707, 0.31055993,
       0.03215153, 0.29152378, 0.16666147, 0.03419349, 0.0322932 ,
       0.51841253, 0.19386658, 0.48419002, 0.04550255, 0.04735994,
       0.0322932 , 0.03419349, 0.06452044, 0.18095762, 0.09562363,
       0.05135094, 0.03419349, 0.32851875, 0.03963808, 0.05198386,
       0.03433517, 0.07830214, 0.08752049, 0.25703555, 0.0324519 ],
      dtype=float32)

In [45]:
pd.DataFrame({"pred":pred.flatten(),"real":Y_valid[0:50]})

,pred,real
0,0.050923,0
1,0.038748,0
2,0.139359,0
3,0.188261,0
4,1.814379,0
5,0.036143,0
6,0.038446,0
7,0.034335,0
8,0.132678,0
9,0.306370,0


We got a very bad result!  Even though it looks ok with the Mean squared error, notice that we still need to iterate to get better results.
So what we are going to do is to keep improving this model to see if we can make it work. Maybe we need some hibrid model like we did in the previous notebook. One model to catch the trend and another to learn this complex interactions.